<a href="https://colab.research.google.com/github/vkrisvasan/llamaKV/blob/main/LearnGrerelAIdataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install datasets -q

In [28]:
from datasets import load_dataset
dataset = load_dataset("gretelai/synthetic_text_to_sql", split = "train")
# Filter for "Retail" domain, select first 50 rows, and choose specific columns
retail_dataset = dataset.filter(lambda example: example['domain'] == 'retail') \
                        .select(range(50)) \
                        .map(lambda example: {'domain': example['domain'],
                                              'domain_description': example['domain_description'],
                                              'sql_prompt': example['sql_prompt'],
                                              'sql': example['sql']})#,
                             #remove_columns=dataset.column_names)  # Remove all original columns
print(retail_dataset)
print(retail_dataset[0])

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'domain', 'domain_description', 'sql_complexity', 'sql_complexity_description', 'sql_task_type', 'sql_task_type_description', 'sql_prompt', 'sql_context', 'sql', 'sql_explanation'],
    num_rows: 50
})
{'id': 5145, 'domain': 'retail', 'domain_description': 'Retail data on circular supply chains, ethical labor practices, product transparency, and consumer education.', 'sql_complexity': 'subqueries', 'sql_complexity_description': 'subqueries, including correlated and nested subqueries', 'sql_task_type': 'analytics and reporting', 'sql_task_type_description': 'generating reports, dashboards, and analytical insights', 'sql_prompt': 'What is the percentage of products that are made from recycled materials?', 'sql_context': 'CREATE TABLE products(product_id INT, is_recycled BOOLEAN);', 'sql': 'SELECT (COUNT(*) * 100.0 / (SELECT COUNT(*) FROM products)) as percentage FROM products WHERE is_recycled = TRUE;', 'sql_explanation': 'This query calculates the percenta

In [29]:
# Iterate over the dataset and print each row in a readable format
for i, row in enumerate(retail_dataset):
    print(f"Row {i+1}:")
    for key, value in row.items():
        print(f"  {key}: {value}")
    print("-" * 20)

Row 1:
  id: 5145
  domain: retail
  domain_description: Retail data on circular supply chains, ethical labor practices, product transparency, and consumer education.
  sql_complexity: subqueries
  sql_complexity_description: subqueries, including correlated and nested subqueries
  sql_task_type: analytics and reporting
  sql_task_type_description: generating reports, dashboards, and analytical insights
  sql_prompt: What is the percentage of products that are made from recycled materials?
  sql_context: CREATE TABLE products(product_id INT, is_recycled BOOLEAN);
  sql: SELECT (COUNT(*) * 100.0 / (SELECT COUNT(*) FROM products)) as percentage FROM products WHERE is_recycled = TRUE;
  sql_explanation: This query calculates the percentage of products that are made from recycled materials by dividing the count of recycled products by the total number of products.
--------------------
Row 2:
  id: 5352
  domain: retail
  domain_description: Retail data on circular supply chains, ethical la